# Checking the model

## Initialization
There is only one parameter to enter here, the key you got from the instructions. It will allow to decrypt the data needed to access the content of the bucket where the model is stored.

In [1]:
# Enter the key here (keep the quotes around it)
key = ''

## You can now execute all the cells in the notebook

In [2]:
# Install needed libraries
!pip install -q cryptography


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# Decrypt and set some parameters
from cryptography.fernet import Fernet
f = Fernet(key)
with open('connection', 'r') as file:
    # Read all lines into a list variable
    lines = file.readlines()
aws_access_key_id = f.decrypt(lines[0]).decode('utf-8')
aws_secret_access_key = f.decrypt(lines[1]).decode('utf-8')

# Don't change the bucket name
bucket_name = 'rh-mad-workshop-m6'

print("Here are the info you will need in the next section:")
print("----------------------------------------------------")
print(f"Access key: {aws_access_key_id}")
print(f"Secret key: {aws_secret_access_key}")
print(f"Bucket name: {bucket_name}")

Here are the info you will need in the next section:
----------------------------------------------------
Access key: [removed]
Secret key: [removed]
Bucket name: rh-mad-workshop-m6


**Please save these information somewhere or leave this notebook open. We will need them later on.**

## Imports
Of course we will need to use some libraries to access the storage, so import them by running the following cell. Boto3 is the standard library (from AWS) to interact with all their services.

In [4]:
import boto3
import botocore

## S3 Client

Now, let's create the client that will be used for the connection (you can that see we are using the parameters we defined earlier).

In [5]:
s3 = boto3.client('s3',
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key = aws_secret_access_key,
                region_name = 'us-east-1')

## Verify that we can connect and see our bucket

In [6]:
connect = s3.head_bucket(Bucket=bucket_name)
if connect['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Everything seems right, I can connect!")
    print(connect)
else:
    print("Hum, there seems to be a problem...")

Everything seems right, I can connect!
{'ResponseMetadata': {'RequestId': '5V7JQ7WFDV2RWA7B', 'HostId': 'JnMX/CCCBksdx30G8x/OLmDo1pL2l9M0JM9RHqYa3PQnnjmG3vRq0/Ca/YDc0QVw3M8/JiFnlZo=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'JnMX/CCCBksdx30G8x/OLmDo1pL2l9M0JM9RHqYa3PQnnjmG3vRq0/Ca/YDc0QVw3M8/JiFnlZo=', 'x-amz-request-id': '5V7JQ7WFDV2RWA7B', 'date': 'Mon, 11 Dec 2023 20:55:15 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


## Verify the location of the model inside the bucket

In [17]:
response = s3.list_objects(Bucket=bucket_name)

if 'Contents' in response:
    for content in response['Contents']:
        print(content['Key'])
else:
    print("Bucket is empty")
    
# s3.copy("rh-mad-workshop-m6", "coolstore-model/")

# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-download-file.html

s3.download_file("rh-mad-workshop-m6", "coolstore-model/coolstore_3000images_300epochs.onnx", "coolstore_3000images_300epochs.onnx")

coolstore-model/
coolstore-model/coolstore_3000images_300epochs.onnx


## Now 
#### - manually download coolstore_3000images_300epochs.onnx to your laptop from Jupyter where it should now be
#### - Create a Minio implemenation in a new project called 'minio'
#### - Upload coolstore_3000images_300epochs.onnx to a public bucket called 'models' in Minio

## Download Model from Minio Object Storage

Perfect, we can see that the file is located inside the **coolstore-model** folder!

You can now get back to the instructions.

In [22]:
!pip install minio


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [23]:
from minio import Minio


In [24]:
def get_s3_server():
    minioClient = Minio('minio-service.minio.svc.cluster.local:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)

    return minioClient
            

In [31]:
def download_models(minio_client, s3_folder_path):
    # not tested code
    # it will get all the files from the user bucket
    # models = minio_client.list_objects("models", recursive=True, prefix=f"/{s3_folder_path}")
    models = minio_client.list_objects("models", recursive=True)
    print(models)
    for model in models:
        print(model)
        modelname = model.object_name
        print(modelname)
        minio_client.fget_object('models', modelname, "/tmp/"+modelname)

In [35]:
minioClient = get_s3_server()

download_models(minioClient, "")

# model_file = open("/tmp/coolstore_3000images_300epochs.onnx", "r")

# print(model_file.__sizeof__())

<generator object Minio._list_objects at 0x7f156014fba0>
coolstore_3000images_300epochs.onnx


In [36]:
import os

file_stats = os.stat("/tmp/coolstore_3000images_300epochs.onnx")

print(file_stats)
print(f'File Size in Bytes is {file_stats.st_size}')
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

os.stat_result(st_mode=33188, st_ino=38099189, st_dev=2097218, st_nlink=1, st_uid=1000720000, st_gid=0, st_size=83924139, st_atime=1702356824, st_mtime=1702356825, st_ctime=1702356825)
File Size in Bytes is 83924139
File Size in MegaBytes is 80.0362958908081
